# Chapter 1

**Set up a basic RAG pipeline (BM25/TFIDF + simple QA model)**

In this chapter, we will learn about building a simple RAG pipeline. We will mainly focus on how to preprocess and chunk the data followed by building a simple retrieval engine without using any fancy "Vector Index". The idea is to show the inner working of a retrieval pipeline and make you understand the workflow from a user query to a generated response using an LLM.

For this chapter you will need a Cohere API key.

Create an `.env` file in the `notebooks` directory. The file's content is:

```
CO_API_KEY="<your-cohere-api-key>"
```
`.env` combined with `dotenv` allows for better API keys management in notebooks.

In [ ]:
import json
import os
import pathlib
from datetime import datetime
from typing import Dict, List

import dotenv
import numpy as np
import wandb
import cohere
from scipy.spatial.distance import cdist
from sklearn.feature_extraction.text import TfidfVectorizer


dotenv.load_dotenv()

Here, we will start a Weights and Biases (W&B) run.

Throughout this notebook, W&B will be used to store, version, and download text files, creating a lineage. We will begin with an already uploaded raw data file as a [W&B Artifact](https://docs.wandb.ai/guides/artifacts), download it, inspect it, and devise a chunking strategy. Finally, we will upload the processed documents back to W&B as an Artifact, establishing a lineage (DAG) between the raw and processed data.

In [ ]:
WANDB_ENTITY = "rag-course"
WANDB_PROJECT = "dev"

wandb.require("core")

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    group="Chapter 1",
)

In [ ]:
# TODO: Remove this once we more to the final project
# documents_artifact = wandb.Artifact(
#     name="wandb_docs",
#     type="dataset",
#     description="W&B Documentation in Markdown format",
#     metadata={
#         "total_files": 380,
#         "date_processed": datetime.now().strftime("%Y-%m-%d"),
#     },
# )

# documents_artifact.add_dir("../data/wandb_docs")
# run.log_artifact(documents_artifact)

## Data ingestion

### Loading the data

Use [W&B Artifacts](https://docs.wandb.ai/guides/artifacts) to track and version data as the inputs and outputs of your W&B Runs. For example, a model training run might take in a dataset as input and produce a trained model as output. W&B Artifact is a powerful object storage with rich UI functionalities.

Below we are downloading an artifact named `wandb_docs` which will download 380 markdown files in your `../data/wandb_docs` directory. This is our data source.

In [ ]:
documents_artifact = run.use_artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/wandb_docs:latest", type="dataset"
)
data_dir = "../data/wandb_docs"

docs_dir = documents_artifact.download(data_dir)

Upon inspecting the `../data/wandb_docs` directory below, we see that we have downloaded 380 files. The first 5 files are all in markdown (`.md` file format).

In [ ]:
docs_dir = pathlib.Path(docs_dir)
docs_files = sorted(docs_dir.rglob("*.md"))

print(f"Number of files: {len(docs_files)}\n")
print("First 5 files:\n{files}".format(files="\n".join(map(str, docs_files[:5]))))

Lets look at an example file below. We take the first element of the list (`docs_files`) and use a convenient `Path.read_text` method which returns the decoded contents of the pointed-to file as a string.

💡 Looking at the example, we see some format to it. While building an ingestion pipeline, it is a good practice to look through few examples to see if there is any pattern to your data source. It helps to come up with better preprocessing steps and chunking strategies.

In [ ]:
print(docs_files[0].read_text())

Below, we are storing files as dictionaries with content (raw text) and metadata. Metadata is extra information for that data point which can be used to group together similar data points, or filter out a few data points. We will see in future chapters the importance of metadata and why it should not be ignored while building the ingestion pipeline.

The metadata can be derived (`raw_tokens`) or is inherent (`source`) to the data point.

Note that we are simply doing word counting and calling it `raw_tokens`. In practice we would be using the [tiktoken tokenizer](https://github.com/openai/tiktoken) to calculate the token counts but this naive calculation is an okay approximation for now.

In [ ]:
# We'll store the files as dictionaries with some content and metadata
data = []
for file in docs_files:
    content = file.read_text()
    data.append(
        {
            "content": content,
            "metadata": {
                "source": str(file.relative_to(docs_dir)),
                "raw_tokens": len(content.split()),
            },
        }
    )
data[:2]

Checking the total number of tokens of your data source is a good practice. In this case, the total tokens is more than 200k. Surely, most LLM providers cannot process these many tokens. Building a RAG is justified in such cases.

In [ ]:
total_tokens = sum(map(lambda x: x["metadata"]["raw_tokens"], data))
print(f"Total Tokens in dataset: {total_tokens}")

The newly created list of dictionaries with `content` and `metadata` will now be logged as a W&B Artifact called `raw_data`. We started with the `wandb_docs` artifact and now we are are logging the processed data as `raw_data` artifact.

In [ ]:
# Let's store the raw data in an artifact for future use and reproducibility
raw_artifact = wandb.Artifact(
    name="raw_data",
    type="dataset",
    description="Raw wandb documentation",
    metadata={
        "total_files": len(data),
        "date_processed": datetime.now().strftime("%Y-%m-%d"),
        "total_raw_tokens": total_tokens,
    },
)
with raw_artifact.new_file("documents.jsonl", mode="w") as f:
    for item in data:
        f.write(json.dumps(item) + "\n")
run.log_artifact(raw_artifact)

### Chunking the data

Each document contains a large number of tokens, so we need to split it into smaller chunks to manage the token count per chunk. This approach serves three main purposes:

* Most embedding models have a limit of 512 tokens per input (based on the tokenizer used during their training).

* Chunking allows us to retrieve and send only the most relevant portions to our LLM, significantly reducing the total token count. This helps keep the LLM’s cost and processing time manageable.

* When the text is small-sized, embedding models tend to generate better vectors as they can capture more fine-grained details and nuances in the text, resulting in more accurate representations.

Below we are chunking each content (text) to a maximum length of 500 tokens (`CHUNK_SIZE`). We are not overlapping (`CHUNK_OVERLAP`) the content of one chunk with another chunk.


In [ ]:
# These are hyperparameters of our ingestion pipeline

CHUNK_SIZE = 300
CHUNK_OVERLAP = 0


def split_into_chunks(
    text: str, chunk_size: int = CHUNK_SIZE, chunk_overlap: int = CHUNK_OVERLAP
) -> List[str]:
    """Function to split the text into chunks of a maximum number of tokens
    ensure that the chunks are of size CHUNK_SIZE and overlap by chunk_overlap tokens
    use the `tokenizer.encode` method to tokenize the text
    """
    tokens = text.split()
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk = tokens[start:end]
        chunks.append(" ".join(chunk))
        start = end - chunk_overlap
    return chunks

We will use the `raw_data` artifact we pushed as Artifacts earlier. Let's download it. By using `use_artifact` method, we are starting a lineage from the `raw_data` artifact to the new artifact we will create further below.

In [ ]:
raw_artifact = run.use_artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/raw_data:latest", type="dataset"
)
artifact_dir = raw_artifact.download()
raw_data_file = pathlib.Path(f"{artifact_dir}/documents.jsonl")
raw_data = list(map(json.loads, raw_data_file.read_text().splitlines()))
raw_data[:2]

Let us chunk each document in the raw data Artifact. We create a new list of dictionaries with the chuked text (`content`) and with `metadata`.

In [ ]:
chunked_data = []
for doc in raw_data:
    chunks = split_into_chunks(doc["content"])
    for chunk in chunks:
        chunked_data.append(
            {
                "content": chunk,
                "metadata": {
                    "source": doc["metadata"]["source"],
                    "raw_tokens": len(chunk.split()),
                },
            }
        )

### Cleaning the data

We clean the chunks for special tokens that we find breaks the OpenAI's `client.chat.completions` api. The data cleaning step is crucial for most ML pipelien and even for a RAG/Agentic pipeline. Usually, higher quality chunks provided to an LLM generates a higher quality response.


In [ ]:
def make_text_tokenization_safe(content: str) -> str:
    special_tokens_set = {
        "<|endofprompt|>",
        "<|endoftext|>",
        "<|fim_middle|>",
        "<|fim_prefix|>",
        "<|fim_suffix|>",
    }

    def remove_special_tokens(text: str) -> str:
        """Removes special tokens from the given text.

        Args:
            text: A string representing the text.

        Returns:
            The text with special tokens removed.
        """
        for token in special_tokens_set:
            text = text.replace(token, "")
        return text

    cleaned_content = remove_special_tokens(content)
    return cleaned_content

In [ ]:
cleaned_data = []
for doc in chunked_data:
    cleaned_doc = doc.copy()
    cleaned_doc["cleaned_content"] = make_text_tokenization_safe(doc["content"])
    cleaned_doc["metadata"]["cleaned_tokens"] = len(
        cleaned_doc["cleaned_content"].split()
    )
    cleaned_data.append(cleaned_doc)
cleaned_data[:2]

Again we will store the cleaned data as an Artifact named `chunked_data`. The metadata that we are logging along with the artifact allows us to go back to it later and be able to reproduce the cleaning steps. It also allows us to pick the version of the `chunked_data` that we are willing to experiment with.

In [ ]:
total_raw_tokens = sum(map(lambda x: x["metadata"]["raw_tokens"], cleaned_data))
total_cleaned_tokens = sum(map(lambda x: x["metadata"]["cleaned_tokens"], cleaned_data))

chunked_artifact = wandb.Artifact(
    name="chunked_data",
    type="dataset",
    description="Chunked wandb documentation",
    metadata={
        "total_files": len(cleaned_data),
        "date_processed": datetime.now().strftime("%Y-%m-%d"),
        "total_raw_tokens": total_raw_tokens,
        "total_cleaned_tokens": total_cleaned_tokens,
        "chunk_size": CHUNK_SIZE,
        "chunk_overlap": CHUNK_OVERLAP,
    },
)
with chunked_artifact.new_file("documents.jsonl", mode="w") as f:
    for item in cleaned_data:
        f.write(json.dumps(item) + "\n")
run.log_artifact(chunked_artifact)

## Vectorizing the data

One of the key ingredient of most retrieval system is to represent the given modality (text in our case) as a vector. This vector is a numerical representation representing the "content" of that modality (text). 

Text vectorization (text to vector) can be done using various techniques like [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model), [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf) (Term Frequency-Inverse Document Frequency), and embeddings like [Word2Vec](https://en.wikipedia.org/wiki/Word2vec), [GloVe](https://nlp.stanford.edu/projects/glove/), and transformer based architectures like BERT and more, which capture the semantic meaning and relationships between words or sentences. 

Below, we are downloading the `cleaned_data` artifact.

In [ ]:
chunked_artifact = run.use_artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/chunked_data:latest", type="dataset"
)
artifact_dir = chunked_artifact.download()
chunked_data_file = pathlib.Path(f"{artifact_dir}/documents.jsonl")
chunked_data = list(map(json.loads, chunked_data_file.read_text().splitlines()))
chunked_data[:2]

![01_lineage_artifacs_data](../images/01_lineage_artifacts_data.png)

Below we are creating a simple `Retriever` class. This class is responsible for vectorizing the chunks using the `index_data` method and provides a convenient method `search`, for querying the vector index using cosine distance similarity.

- `index_data` will take a list of chunks and vectorize it using TF-IDF and store it as `index`. 

- `search` will take a `query` (question) and vectorize it using the same technique (TF-IDF in our case). It then computes the cosine distance between the query vector and the index (list of vectors) and pick the top `k` vectors from the index. These top `k` vectors represent the chunks that are closest (most relevant) to the `query`.

---

Note that the `Retriever` class is inherited from `weave.Model`. TODO: a bit on pydantic `BaseModel`?

A Model is a combination of data (which can include configuration, trained model weights, or other information) and code that defines how the model operates. By structuring your code to be compatible with this API, you benefit from a structured way to version your application so you can more systematically keep track of your experiments.

To create a model in Weave, you need the following:

- a class that inherits from weave.Model
- type definitions on all attributes
- a typed `predict`, `infer` or `forward` method with `@weave.op()` decorator.

Imagine `weave.op()` to be a shameless use of `print` statement. If you have not initialized a weave run by doing `weave.init`, the code will work as it is without any tracking.

The `predict` method decodated with `weave.op()` will track the model settings along with the inputs and outputs anytime you call it.

In [ ]:
import weave

In [ ]:
class Retriever(weave.Model):
    vectorizer: TfidfVectorizer = TfidfVectorizer()
    index: list = None
    data: list = None

    @weave.op()
    def index_data(self, data):
        self.data = data
        docs = [doc["cleaned_content"] for doc in data]
        self.index = self.vectorizer.fit_transform(docs)

    @weave.op()
    def search(self, query, k=5):
        query_vec = self.vectorizer.transform([query])
        cosine_distances = cdist(
            query_vec.todense(), self.index.todense(), metric="cosine"
        )[0]
        top_k_indices = cosine_distances.argsort()[:k]
        output = []
        for idx in top_k_indices:
            output.append(
                {
                    "source": self.data[idx]["metadata"]["source"],
                    "text": self.data[idx]["cleaned_content"],
                    "score": 1 - cosine_distances[idx],
                }
            )
        return output

    @weave.op()
    def predict(self, query: str, k: int):
        return self.search(query, k)

Let's see our Retriever in action. We will index our chunked data and then ask a question to retriev related chunks.

We will not be using weave here to show that the code works as it is and that weave is a lightweight wrapper, the benefit of which we will show in the later sections.

In [ ]:
retriever = Retriever()
retriever.index_data(chunked_data)

In [ ]:
query = "How do I use W&B to log metrics in my training script?"
search_results = retriever.search(query)
for result in search_results:
    print(result)

## Generating a response

There are two components of any RAG pipeline - a `Retriever` and a `ResponseGenerator`. Earlier, we designed a simple retriever. Here we are designing a simple `ResponseGenerator`. 

The `generate_response` method takes the user question along with the retrieved context (chunks) as inputs and makes a LLM call using the `model` and `prompt` (system prompt). This way the generated answer is grounded on the documentation (our usecase). In this course we are using Cohere's `command-r` model.

As earlier, we have wrapped this `ResponseGenerator` class with weave for tracking the inputs and the output.

In [ ]:
class ResponseGenerator(weave.Model):
    model: str
    prompt: str
    client: cohere.Client = None
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.client = cohere.Client(api_key=os.environ["CO_API_KEY"])

    @weave.op()
    def generate_context(self, context: List[Dict[str, any]]) -> str:
        return [{"source": item['source'], "text": item['text']} for item in context]
    
    @weave.op()
    def generate_response(self, query: str, context: List[Dict[str, any]]) -> str:
        contexts = self.generate_context(context)
        response = self.client.chat(
            preamble=self.prompt,
            message=query,
            model=self.model,
            documents=contexts,
            temperature=0.1,
            max_tokens=2000,
        )
        return response.text

    @weave.op()
    def predict(self, query: str, context: List[Dict[str, any]]):
        return self.generate_response(query, context)

Below is the system prompt. Consider this to be set of instructions on what to do with the user question and the retrieved contexts. In practice, the system prompt can be very detailed and involved (depending on the usecase) but we are showing a simple prompt. Later we will iterate on it and show how improving the system prompt improves the quality of the generated response.

In [ ]:
PROMPT = """
Answer to the following question about W&B. Provide an helful and complete answer based only on the provided documents.
"""

Let's generate the response for the question "How do I use W&B to log metrics in my training script?". We have already retrieved the context in the previous section and passing both the question and the context to the `generate_response` method.

In [ ]:
response_generator = ResponseGenerator(model="command-r", prompt=PROMPT)
answer = response_generator.generate_response(query, search_results)
print(answer)

## Simple Retrieval Augmented Generation (RAG) Pipeline

Below we are bringing everything together. As stated a simple RAG pipeline constitute of a retriever and a response generator. 

The `__call__` method is calling the `predict` method which is decorated with `weave.op()`. It takes the user query, retrieves relevant context using the retriever and finally synthesize a response grounded on the data source (documentation in our case).

In [ ]:
class RAGPipeline(weave.Model):
    retriever: Retriever = None
    response_generator: ResponseGenerator = None
    top_k: int = 5

    @weave.op()
    def predict(self, query: str):
        context = self.retriever.predict(query, self.top_k)
        return self.response_generator.predict(query, context)

Let us initialize the `RAGPipeline`.

In [ ]:
# Initialize retriever
retriever = Retriever()
retriever.index_data(chunked_data)

# Initialize the response generator
response_generator = ResponseGenerator(model="command-r", prompt=PROMPT)

# Bring them together as a RAG pipeline
rag_pipeline = RAGPipeline(
    retriever=retriever,
    response_generator=response_generator,
    top_k=5
)

We are finally ready to use `weave.init(<project-name>)` to see what we get from using W&B Weave. Once intialized, we will start tracking the inputs and the outputs along with underlying attributes (model name, top_k, etc.).

In [ ]:
weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

In [ ]:

response = rag_pipeline.predict("How do I get get started with wandb?")
print(response, sep="\n")

Click on the link starting with a 🍩. This is the trace timeline for all the executions that happened in our simple RAG application. Go to the link and drill down to find everything that got tracked.

![weave trace timeline](../images/01_weave_trace_timeline.png)

In [ ]:
# TODO: Add exercise for chapter 1.